In [106]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from matplotlib import pyplot

# Women in engineering, science, mathematics and technologie - Undertaking a Benchmarking for the "Deutsche Forschungsgesellschaft" with Sweden

By Maika Krüger, 
FU Berlin,
Bsc Computer Science

## Introduction

The DFG (Deutsche Forschnungsgesellschaft) focuses especially on equal opportunities and gender equality (GE) for (junior) scientist*. By undertaking a Benchmarking, DFG would like to know their state of the art on this topic focusing on Engineering,Natural Science, Mathematics and Technologie (STEM). Having the highest Score of the "Gender Equality Index" in the European Union (EU) Sweden is choosen as the Benchmark.

*Quelle:https://www.dfg.de/dfg_profil/aufgaben/was_ist_die_dfg/index.html

Annotation:

In the following I will just focus on current Projects of the DFG. These are Projects not finished in 2018. Moreover I will only look at STEM-fields.

## Gender Equality Index 

In [104]:
# Load Data
gender = pd.read_excel('genderindex.xls', skiprows=1)
# Quelle: https://eige.europa.eu/gender-equality-index/2019

handmade:

1) Copied from excel file into a single sheet 2017.

In [103]:
gender = gender.set_index('Country')
k = gender.Gender_EqualityIndex.sort_values(ascending=False)
idx  = k.index

# visualize
colors = ['#FEBFB3']*29
colors[0] = 'crimson'
colors[11] = 'crimson'
colors[12] = 'crimson'

fig = go.Figure(data=[
    go.Bar(x=idx, y=k, marker_color = colors),
])
# Change the bar mode
fig.update_layout(barmode='group', title = 'Gender Equality Index of the EU (2017)',yaxis=dict(range=[0, 100]), yaxis_title = 'Gender Equality Index / Points')
fig.show()

In this graph one can see the Gender Equality Index of the EU-Countries in 2017. Sweden has the highest Score (83,6). Germany has a Score of 66,9, which is less than the EU-mean of 67,4. Therefore Sweden is chosen as the Benchmark by the DFG.

Besides Sweden decribes itself as the fist feminism government.*

*Quelle:https://sweden.se/society/gender-equality-in-sweden/

In [4]:
# Prepare the Gepris Data

# Load Gepris Data
project = pd.read_csv('extracted_project_data.csv')# information about the Projects
person = pd.read_csv('extracted_person_data.csv')# Professors/ Dokotors
relation_pp = pd.read_csv('project_person_relations.csv')# relation Project and Person
subject = pd.read_csv('subject_areas.csv')# subjectareas fundet by DFG
subjectareas = pd.read_csv('project_ids_to_subject_areas.csv')# subjectareas of the projects

# Filter for male and female Professors on the Gepris Data
prof = person[person['name'].str.contains(pat = 'Professor|Professorin' )]# select all professors
prof_f = prof[prof['name'].str.contains(pat= 'Professorin')]# select female Profs
prof_m = prof[prof['name'].str.contains(r'\bProfessor\b')]# select male Profs

# Female
# Join Persons and Projects and Subjects, Note: One Professor can have several Projects
join_project_realtion_pp  = relation_pp.merge(project, left_on ='project_id_number', right_on = 'project_id_number')
femaleproject = pd.merge(prof_f, join_project_realtion_pp, left_on = 'person_id', right_on = 'person_id', how = 'left')# Projects of female Profs
femalesubject = pd.merge(femaleproject, subjectareas, left_on = 'project_id_number', right_on = 'project_id', how = 'left')# Projects joined with Subject areas of female Profs
#print('The total amount of Projects of female Professors is:',len(femalesubject))# one project can have several subjectareas

# Female
# Filter for Current Projects of female Professors. 
# Current Projects are Projects which are not finished yet.
current = femalesubject[femalesubject['funding_end_year'].isnull()] # filter for current projects
grouped_current = current.groupby(['subject_area']).size().sort_values(ascending=False)# Group projects per subject area
idx = grouped_current.sort_values(ascending=False).index # count amount of projects per subjects and set it as an index
grouped_current = grouped_current.loc[idx]
#print('The amount of current projects of females Professors is:',len(current))
#print('The current 5 most poular subjectareas wfP are:', grouped_current.head())

# Female
# Filter Procjets of female Professors in STEM in the Gepris Data 
stem0 = pd.merge(subject, femalesubject, left_on ='subject_area', right_on = 'subject_area', how = 'left')
stem = stem0[stem0['research_area'].str.contains(pat = 'Ingenieurwissenschaften|Naturwissenschaften|Mathematik')]#select all professors

# Male
# Filter Procjets, Subjects of male Professors in STEM in the Gepris Data 
men_projects = pd.merge(prof_m, join_project_realtion_pp, left_on = 'person_id', right_on = 'person_id', how= 'left')
men_subject = pd.merge(men_projects, subjectareas, left_on = 'project_id_number', right_on = 'project_id', how = 'left')

# Male
# Filter Projects of male Professors in STEM in the GEPRIS Data
men_stem0 = pd.merge(subject, men_subject, left_on = 'subject_area', right_on = 'subject_area', how = 'left')
men_stem = men_stem0[men_stem0['research_area'].str.contains(pat='Ingenieurwissenschaften|Naturwissenschaften|Mathematik')]

## Current projects funded by DFG

In [5]:
# Prepare Gepris Data

# Filter Current Projects. These are Projects not beeing finished in 2018
stateoftheart_f = stem[stem['funding_end_year'].isnull()] # filter for current projects in stem areas of female Professors
stateoftheart_m = men_stem[men_stem['funding_end_year'].isnull()] # filter for current projects in stem areas of male Professors

# total amount of current projects
total = len(stateoftheart_f) + len(stateoftheart_m)
#print('Total current Projects:',total)

In [6]:
sex=['male', 'female']
colours = ['#96D38C','#E1396C']
amount = [len(stateoftheart_m), len(stateoftheart_f)]

fig = go.Figure([go.Bar(x=sex, y = amount, marker_color=colours)])
fig.update_layout(title='.                           current DFG-projects in STEM with respect to professor’s sex',
                 xaxis_title='professor’s sex', yaxis=dict(range=[0, 12000]),
                 yaxis_title='amount of projects')
fig.show()

Current projects are projects which have not been finished in 2018. As I already said we will just focus at Projects in STEM-fields.
By looking at the bar chart, one sees a huge differenece of the amount of projects lead by male and by female professors. The total amount of current projects is 12565 where 1254 are lead by female professors and 11311 are lead by male professors. From this it follows that about 10% of the projects funded by DFG are managed by female professors.

## Current ratio of the sex of professors funded by DFG

In [7]:
# prepare the GEPRIS Data

# female
# join current projects with female Professors
current_f = pd.merge(prof_f, stateoftheart_f, left_on = 'person_id', right_on = 'person_id')
current_f_proj = current_f # current female projects
# group_proff = current_f.groupby(['person_id']).size() other idea tto get female profs and to remove their dublicates

# female
# female professors having a running project in STEM fields
current_f.sort_values("person_id", inplace = True) # sorting by person_id
current_f.drop_duplicates(subset ="person_id", keep = 'first', inplace = True) # dropping ALL duplicte person ids
current_f.sort_values("name_x", inplace = True) # sort by name
#print('The amount of the female Professors in STEM-areas getting fundings for current projects by the DFG are:',len(current_f ))

# male
# join current projects with male professors
current_m = pd.merge(prof_m, stateoftheart_m, left_on = 'person_id', right_on = 'person_id')
current_m_proj = current_m # current male projects
# group_profm = current_m.groupby(['name_x']).size() other idea to get male profs and to drop their dublicates

# male
# male professors having a running project in STEM fields
current_m.sort_values("person_id", inplace = True) # sorting by person_id
current_m.drop_duplicates(subset ="person_id", keep = 'last', inplace = True) # dropping ALL duplicte person ids
current_m.sort_values("name_x", inplace = True) # sorting by name
#print('The amount of the male Professors in STEM-areas getting fundings for current projects by the DFG are:',len(current_m ))

In [8]:
colors = ['#E1396C', '#96D38C']
#colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

fig = go.Figure(data=[go.Pie(labels=['DFG female professors','DFG male professors'],
                             values=[len(current_f),len(current_m)])])
fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update_layout(title = '.            ratio of professor´s sex leading current DFG-projects in STEM')
fig.show()

The pie chart gives a good expression of the ratio of female and male professors  currently leading a project in STEM which is funded by DFG. 100% is the total amount of professors. Most of the professors are male professors which is 88%. Only 12% are female professors.

## Top 10 subjectareas of female professors in STEM funded by DFG

In [9]:
# prepare Gepris Data
areas = stateoftheart_f.loc[:,['subject_area']] # select female subjectareas
grouped_areas = areas.groupby(['subject_area']).size().sort_values(ascending=False)# group projetcs to subjects
idx = grouped_areas.sort_values(ascending=False).index # count amount of projects per subjects and set it as an index
grouped_areas = grouped_areas.loc[idx]

In [10]:
index_list = idx.tolist()

# translate index to english
labels = {'Mathematik':'mathematics', 'Experimentelle Physik der kondensierten Materie':'experimental physics', 'Theoretische Physik der kondensierten Materie':'theoretical physics',
          'Statistische Physik, Weiche Materie, Biologische Physik, Nichtlineare Dynamik':'statistical physics, biophysics', 'Astrophysik und Astronomie':'astrophysics',
          'Kern- und Elementarteilchenphysik, Quantenmechanik, Relativitätstheorie, Felder':'quantum mechanics',
         'Herstellung und Eigenschaften von Funktionsmaterialien':'material engineering',
         'Analytik, Methodenentwicklung (Chemie)':'analytical chemistry',
         'Geologie, Ingenieurgeologie, Paläontologie':'geology & paleontology',
       'Physikalische Chemie von Molekülen, Flüssigkeiten und Grenzflächen - Spektroskopie, Kinetik':'physical chemistry'}

for x in labels:
    index_list[index_list.index(x)]=labels[x]

In [97]:
fig0 = go.Figure([go.Bar(x = index_list, y = grouped_areas[:10], marker_color='#E1396C')])
fig0.update_layout(title='.                           top 10 subjectareas of DFG-project lead by female professors',
                 yaxis_title='amount of projects')
fig0.show()

Mathematics is the most popular subject it has over 100% more projects lead by female professors than the second ranced subject.
Besides, 9 of the Top 10 subjectareas are in the field of natural sciences, mathematics and statistaics.

## Spendings of the DFG

In [13]:
# load Data
ing = pd.read_excel('dfg-foerderatlas2015_ing.xls', skiprows = 2)
nat = pd.read_excel('dfg-foerderatlas2015_nat.xls', skiprows = 2)
#Quelle:https://www.dfg.de/download/pdf/dfg_im_profil/zahlen_fakten/foerderatlas/2015/dfg_foerderatlas_2015.pdf
#Bundesministerium für Bildung und Forschung (BMBF): Direkte FuE-Projektförderung des Bundes 2011 bis 2013 (Projektdatenbank PROFI).
#Deutsche Forschungsgemeinschaft (DFG): DFG-Bewilligungen für 2011 bis 2013.
#EU-Büro des BMBF: Beteiligungen am 7. EU-Forschungsrahmenprogramm (Laufzeit: 2007 bis 2013, Projektdaten mit Stand 21.02.2014).
#Berechnungen der DFG.

handmade:

1) Split column 'Art der Einrichtung' in two cells

In [14]:
# set 'Art der Einrichtung' as index
ing = ing.set_index('Art_der_Einrichtung') # engineering
nat = nat.set_index('Art_der_Einrichtung') # natural science, mathematics

# filter fundings of the DFG for STEM Fields
ing_gesamt = ing[(ing.index == 'Insgesamt')]
ing_cost = ing_gesamt['Mio.€'].sum() # fundings of engineering
nat_gesamt = nat[(nat.index =='Insgesamt')]
nat_cost = nat_gesamt['Mio.€'].sum() # funings of natural science, mathematics

In [95]:
# visualize
x = ['engineering, computer science, technology','natural science, mathematics']
y1 = [ing_cost, nat_cost]

fig = go.Figure(data=[
    go.Bar(name = 'DFG',x =x, y=y1, marker_color = 'crimson'),
    #go.Bar(name = 'FuE-Projects',x=x, y=y2, marker_color = '#FEBFB3')
])
# Change the bar mode
fig.update_layout(barmode='group', title = '.         average DFG’s spendings on higher education institution in STEM (2011-2013)', yaxis_title = 'spendings / mio €')
#fig.update_layout(barmode='stack')
fig.show()

DFG invested more in mathematics and natural science than in engeenering, computer science and Technology in 2011-2013.
The DFG invested 1.486 Mio € in engineering, computer science and technology. DFG invested 1.679 mio € in natural science and mathematics.

## Education in STEM in Germany

In [16]:
germ = pd.read_csv('EAG_GRAD_ENTR_FIELD_09122019180809455.csv')
#Quelle: https://stats.oecd.org/Index.aspx?datasetcode=EAG_GRAD_ENTR_FIELD#
germ.tail()

#Drop columns which are not needed
germ.drop(['PowerCode Code', 'PowerCode', 'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags', 'COUNTRY', 'Unit Code', 'Unit', 'INDICATOR', 'Indicator', 'YEAR'], axis = 1, inplace = True)

#Filter STEM-FIELDS
stem_germ = germ[germ['Field'].str.contains(r'\bNatural sciences, mathematics and statistics|Information and Communication Technologies|Engineering, manufacturing and construction\b')]
stem_germ.head()

#choosen level L8, L6, L7 of the International Standard Classification of Education (ISCED)
all_germ = stem_germ[stem_germ['ISC11_LEVEL_CAT'].str.contains(r'\bL8|L7|L6\b')]


# filter phd and professors
phd_germ = stem_germ[stem_germ['ISC11_LEVEL_CAT'].str.contains(r'\bL8\b')]

# filter bachelor
ba_germ = all_germ[all_germ['ISC11_LEVEL_CAT'].str.contains(r'\bL6\b')]

# filter master
ma_germ = all_germ[all_germ['ISC11_LEVEL_CAT'].str.contains(r'\bL7\b')]

#set sex as index
index_sex_germ = phd_germ.set_index('Sex') # set sex as index
index_sex_bachelor_germ = ba_germ.set_index('Sex') # set sex as index
index_sex_master_germ = ma_germ.set_index('Sex') # set sex as index

#professor or phd female or male
men_germ = index_sex_germ[index_sex_germ.SEX=='M'] # filter male persons
women_germ = index_sex_germ[index_sex_germ.SEX == 'F'] # filter female persons

#master female or male
men_master_germ = index_sex_master_germ[index_sex_master_germ.SEX=='M'] # filter male persons
women_master_germ = index_sex_master_germ[index_sex_master_germ.SEX == 'F'] # filter female persons

#bachelor female or male
men_bachelor_germ = index_sex_bachelor_germ[index_sex_bachelor_germ.SEX=='M'] # filter male persons
women_bachelor_germ = index_sex_bachelor_germ[index_sex_bachelor_germ.SEX == 'F'] # filter female persons

# visualization
colors = ['#E1396C', '#FEBFB3','#E1396C', '#D0F9B1'] 
labels = ['mathematics, natural sciences, statistics', 'computer science, technology', 'engineering']

fig = go.Figure(data=[
    go.Bar(name='male bachelor degree', x=labels, y=men_bachelor_germ.Value, marker_color = '#96D38C'),
    go.Bar(name='female bachelor degree', x=labels, y=women_bachelor_germ.Value, marker_color = '#E1396C'),
    go.Bar(name='male master degree', x=labels, y=men_master_germ.Value, marker_color = '#D0F9B1'),
    go.Bar(name='female master degree', x=labels, y=women_master_germ.Value, marker_color = '#FEBFB3')
])
# Change the bar mode
fig.update_layout(barmode='group', title = '.                                 degrees in STEM in Germany (2017)', yaxis_title = 'rel. amount / %')
#fig.update_layout(barmode='stack')
fig.show()

To figure out if germany has to less women  having a bachelors or masters degree in STEM, I plotted the OECD Data for Germany.

This plot shows the percentage of students having bachelors or masters degree in STEM separated by their sex (with respect to all students having a similar dregree in this field and having the same sex).

There is a lack between female and male students in STEM.
At total, 16,469% of the women have a bachelors in STEM and 21,585% of the women have a masters in STEM. At total, about 54% of men have a bachelors in STEM and about 51% of men have a masters in STEM.

Moreover, computer science and technology has the fewest percentage of women.

# Benchmark: Sweden

## Higher education in Sweden in STEM (bachelors, masters, phd and professor)

In [17]:
sweden = pd.read_csv('EAG_GRAD_ENTR_FIELD_09122019182523682.csv')
#Quelle: https://stats.oecd.org/Index.aspx?datasetcode=EAG_GRAD_ENTR_FIELD

#Drop columns which are not needed
sweden.drop(['PowerCode Code', 'PowerCode', 'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags', 'COUNTRY', 'Unit Code', 'Unit', 'INDICATOR', 'Indicator', 'YEAR'], axis = 1, inplace = True)

# Filter STEM-FIELDS
stem_sweden = sweden[sweden['Field'].str.contains(r'\bNatural sciences, mathematics and statistics|Information and Communication Technologies|Engineering, manufacturing and construction\b')]

# Foilter Bachelor, Master, phd and higher educations
# International Standard Classification of Education (ISCE)
all_swe = stem_sweden[stem_sweden['ISC11_LEVEL_CAT'].str.contains(r'\bL8|L7|L6\b')]

#choosen level L8 (Promotionen and higher education) in STEM
promotion_swe = stem_sweden[stem_sweden['ISC11_LEVEL_CAT'].str.contains(r'\bL8\b')]

# Bachelordegree(L6) in STEM
ba_swe = all_swe[all_swe['ISC11_LEVEL_CAT'].str.contains(r'\bL6\b')]

# Masterdegree(L7) in STEM
ma_swe = all_swe[all_swe['ISC11_LEVEL_CAT'].str.contains(r'\bL7\b')]

#set sex as index
index_sex = promotion_swe.set_index('Sex') # set sex as index
index_sex_bachelor = ba_swe.set_index('Sex') # set sex as index
index_sex_master = ma_swe.set_index('Sex') # set sex as index


# sex of L8
men = index_sex[index_sex.SEX=='M'] # filter male persons
women = index_sex[index_sex.SEX == 'F'] # filter female persons

# sex of L7
men_master = index_sex_master[index_sex_master.SEX=='M'] # filter male persons
women_master = index_sex_master[index_sex_master.SEX == 'F'] # filter female persons

# sex of L6
men_bachelor = index_sex_bachelor[index_sex_bachelor.SEX=='M'] # filter male persons
women_bachelor = index_sex_bachelor[index_sex_bachelor.SEX == 'F'] # filter female persons

In [18]:
#Percentage of female and male professors having a L6,L7,L8 degree in Sweden in STEM-fields
colors = ['#E1396C', '#FEBFB3','#E1396C', '#D0F9B1'] 
labels = ['mathematics, natural sciences, statistics', 'computer science, technology', 'engineering']

fig = go.Figure(data=[
    go.Bar(name='male bachelors degree', x=labels, y=men_bachelor.Value, marker_color = '#96D38C'),
    go.Bar(name='female bachelors degree', x=labels, y=women_bachelor.Value, marker_color = '#E1396C'),
    go.Bar(name='male masters degree', x=labels, y=men_master.Value, marker_color = '#D0F9B1'),
    go.Bar(name='female masters degree', x=labels, y=women_master.Value, marker_color = '#FEBFB3'),
    go.Bar(name='male PhD and professors', x=labels, y=men.Value),
    go.Bar(name='female PhD and professors', x=labels, y=women.Value)
])
# Change the bar mode
fig.update_layout(barmode='group', title = '.                                 degrees in STEM in Sweden (2017)', yaxis_title = 'rel. amount / %')
fig.show()

This plot shows the percentage of students having bachelors or masters degree in STEM separated by their sex (with respect to all students having a similar dregree in this field and having the same sex).

The lack between men and women in STEM gets relative smaller the higher their education level in Sweden.

In [96]:
sweden = pd.read_csv('EAG_GRAD_ENTR_FIELD_06122019114704505.csv')
#Quelle: https://stats.oecd.org/Index.aspx?datasetcode=EAG_GRAD_ENTR_FIELD
#Drop columns which are not needed
sweden.drop(['PowerCode Code', 'PowerCode', 'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags', 'COUNTRY', 'Unit Code', 'Unit', 'INDICATOR', 'Indicator', 'YEAR'], axis = 1, inplace = True)

# Filter STEM-FIELDS
stem_sweden = sweden[sweden['Field'].str.contains(r'\bNatural sciences, mathematics and statistics|Information and Communication Technologies|Engineering, manufacturing and construction\b')]

# Foilter Bachelor, Master, Doctor and higher educations
# International Standard Classification of Education (ISCE)
all_swe = stem_sweden[stem_sweden['ISC11_LEVEL_CAT'].str.contains(r'\bL8|L7|L6\b')]

#choosen level L8 (Promotionen and higher education) in STEM
promotion_swe = stem_sweden[stem_sweden['ISC11_LEVEL_CAT'].str.contains(r'\bL8\b')]

index_sex = promotion_swe.set_index('Sex') # set sex as index

# sex of L8
men = index_sex[index_sex.SEX=='M'] # filter male persons
women = index_sex[index_sex.SEX == 'F'] # filter female persons
m = men['Value'].sum()
w = women['Value'].sum()


colors = ['#E1396C', '#96D38C']
#colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

fig = go.Figure(data=[go.Pie(labels=['female','male'],
                             values=[w,m])])
fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update_layout(title = '.                            PhDs and professors in STEM in Sweden (2017)')
fig.show()

This pie chart shwos the ratio of male and female PhD and professors in STEM in Sweden. 32% of the swedisch PhD or professors are female and 68% are male.

## Spendings on higher education institution in Sweden

In [25]:
invest = pd.read_excel('invest_swe.xlsx', skiprows=2)
# current spendings on higher education institutions in R&D in STEM in sweden
# Units: million SEK
# Quelle: http://www.statistikdatabasen.scb.se/pxweb/en/ssd/START__UF__UF0301__UF0301U/UoHDriftLaroAmneFina/

handmade:


Dantenquelle: Current costs for R&D in higher education institutions, SEK millions in current prices by field of science and technology, funding source and every other year


0) Löschen von Fächergruppen, die nicht in STEM-Bereiche fallen

1) Überschriften der Spalten eingefügt

2) Subject als neue Spalte eingefügt und in jeder Zeile das  Subject eingefügt

3) Module in jeder Zeile eingefügt

4) Löschen der Fußnoten

5) Für jedes Modul musste je Modul eine Summe der Ausgaben erstellt werden. Insgesamt halndelt es sich um 296 Zeilen, indenen Zwischensummen gebildet werden mussten.

6) Institutionen auf EU-Ebene oder aus dem Ausland mussten unter "Institutioenen"herausgelöscht werden

In [76]:
invest_RD = invest[(invest.Institutions == 'Total R&D funds')] # get total R&D spendings
invest_RD.set_index('Subject') # set Subject as index

group = invest_RD.groupby(['Subject'])['2017'].sum().sort_values(ascending=False)

#creating a dataframe out of group
df = pd.DataFrame(group)

#seperate spendings for the different subjectareas in STEM
spendings_nat = df[df.index == ' Natural sciences']['2017'].sum()
spendings_mat = df[df.index == 'Mathematics']['2017'].sum()
spendings_eng = df[df.index == 'Engineering']['2017'].sum()
spendings_tech = df[df.index == 'Technology']['2017'].sum()
spendings_com = df[df.index == 'Computerscience']['2017'].sum()

# group spendings since DFG has this groups in their report about their spendings
# 1) mathematics, natural science, statistics
# 2) compuer science, technology, enineering

spend_1 = spendings_nat + spendings_mat # group 1
spend_2 = spendings_com + spendings_tech + spendings_eng # group 2

# create list of spendings for visualizing

spendings = [0,0]
spendings[0] = spend_1
spendings[1] = spend_2

In [87]:
from plotly.subplots import make_subplots
x = ['mathematics, natural sciences, statistics','engineering, computer science, technology']

fig = go.Figure(data=[
    go.Bar(name = 'spendings',x=x, y=spendings, marker_color = 'crimson')])
    
# Change the bar mode
fig.update_layout(barmode='group', title = '.               spendings on higher education institution in STEM in Sweden (2017)')
fig.update_yaxes(title_text="spendings on higher education / mio. SEK")

#fig.update_layout(barmode='stack')
fig.show()

The bar chart shwos the spendings on higher education institutions in sweden in million SEK in 2017.
Sweden spends 6695 mio SEK on mathematics, natural science and statsistis in 2017.
And 6431 mio SEK on engineering, computer science and technology in 2017